In [ ]:
!pip install tokenizers
!pip install tqdm boto3 requests regex sentencepiece sacremoses
!pip install sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 22.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 38.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 5.3 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=ccdacbbf10a2bdd001ce143b0c2faa8a3a7f0afd75a93a8238373bb4a200036c
  Stored in directory: /root/.cache/pip/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score
from google.colab import drive
import random
from sklearn.metrics import mean_squared_error, confusion_matrix, accuracy_score
import time
from sklearn.model_selection import GridSearchCV
import os
import re
import torch
# Load pre-trained BERT model and tokenizer
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import time

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Temporary load data meachanism since the normal way takes way too long

raw_train_data, raw_test_data = pd.read_csv("/content/drive/MyDrive/COMP_551/A3/aclImdb/train_data.csv").drop('Unnamed: 0', axis=1), pd.read_csv("/content/drive/MyDrive/COMP_551/A3/aclImdb/test_data.csv").drop('Unnamed: 0', axis=1)

In [ ]:
raw_test_data = raw_test_data.iloc[:1000, :] # Artificially reducing the test data size here
raw_train_data = raw_train_data.iloc[:1000, :] # Artificially reducing the training data size here

### Using pretrained t-5 based summarizer in the preprocessing

In [ ]:
# With summarizer

from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Input text
input_text = "The quick brown fox jumps over the lazy dog. The dog wakes up and gives chase."

# Encode the input text
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Generate the summary
summary_ids = model.generate(input_ids, num_beams=4, max_length=50, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print(summary)


/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


The quick brown fox jumps over the dog.


In [ ]:
# preprocessing

def preprocess_text(text):
    # Lowercase the text

    # previous_time = time.time()
    text = text.lower()

    # Remove special characters and punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()


    # Encode the input text
    input_ids = tokenizer.encode(
        text,
        add_special_tokens=True,
        max_length=510,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation='longest_first'
    )

    # Generate the summary
    summary_ids = model.generate(input_ids, num_beams=4, max_length=50, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    current_time = time.time()
    # print(f"The time taken was: {current_time-previous_time}")
    return summary

raw_test_data['text'] = raw_test_data['text'].apply(preprocess_text)

In [ ]:
raw_test_data.to_csv("/content/drive/MyDrive/COMP_551/A3/aclImdb/raw_test_data_extra.csv")

In [ ]:
raw_test_data = pd.read_csv("/content/drive/MyDrive/COMP_551/A3/aclImdb/raw_test_data_extra.csv")

### Tokenizing

In [ ]:
# Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def tokenizer_function(input_sequence):
    
    encoded_dict = tokenizer.encode_plus(
        input_sequence,
        add_special_tokens=True,
        max_length=510,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation='longest_first'
    )

    return encoded_dict
    # train_input_ids.append(encoded_dict['input_ids'])
    # train_attention_masks.append(encoded_dict['attention_mask'])


def extract_input_id(encoded_dict):
  return encoded_dict['input_ids']
def extract_attention_mask(encoded_dict):
  return encoded_dict['attention_mask']



In [ ]:
# tokenizing and establishing attention masks for each of the input_texts in the test data

raw_test_data['encoded_dict'] = raw_test_data['text'].apply(tokenizer_function) 
raw_test_data['input_ids'] = raw_test_data['encoded_dict'].apply(extract_input_id)
raw_test_data['attention_mask'] = raw_test_data['encoded_dict'].apply(extract_attention_mask)
test_input_ids = torch.cat(list(raw_test_data['input_ids']), dim=0)
test_attention_masks = torch.cat(list(raw_test_data['attention_mask']), dim=0)
test_labels = torch.tensor(list(raw_test_data['sentiment']))

In [ ]:
# Create a TensorDataset object from the test data
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)

# Create a DataLoader object from the test dataset
test_dataloader = DataLoader(
    test_dataset,
    batch_size=3,
    shuffle=True,
    num_workers=2,
    drop_last=True
)

### Loading the model trained in the Bert Extra Train notbook

In [ ]:
# Load the saved model state dictionary
state_dict = torch.load( '/content/drive/MyDrive/COMP_551/A3/aclImdb/model_extra.pth')

# Load the pre-trained BERT model and tokenizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Add the previously defined head
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(0.2),
    torch.nn.Linear(in_features=model.config.hidden_size, out_features=2)
)

# Load the saved model state dictionary into the model
model.load_state_dict(state_dict)

# Set the model to evaluation mode
model.eval()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

### Evaluating the model on the test data

In [ ]:
# Evaluate the model on test data
total_correct=0
y_pred = []
with torch.no_grad():
    correctly_predicted_attentions=[]
    incorrectly_predicted_attentions=[]
    arr1_size=0
    arr2_size=0
    for batch in test_dataloader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, output_attentions=True)
        predictions = torch.argmax(outputs.logits, dim=1)

        # adding to y_pred
        y_pred+=(predictions)
        

        accuracy = (predictions == labels).float().mean()
        
        if accuracy==1 and arr1_size!=1:
          correctly_predicted_attentions.append((outputs.attentions[0][0], input_ids)) # [block_index][head_index]
          arr1_size+=1

        if accuracy==0 and arr2_size!=1:
          incorrectly_predicted_attentions.append((outputs.attentions[0][0], input_ids)) # [block /_index][head_index]
          arr2_size+=1
        
        
        total_correct+=accuracy


In [ ]:
total_correct/len(raw_test_data)

tensor(0.8010)